In [58]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, learning_curve, validation_curve
import sklearn.model_selection as model_selection

In [59]:
train_data_path = 'data/train.csv'
X = pd.read_csv(train_data_path) 

# What is this method
X = X.loc[:, ~X.columns.str.contains('^Unnamed')]

print(X.shape)
print(X.columns)
X.head()

(202944, 28)
Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income',
       'BMI_Category', 'Healthy_Diet', 'Mental_Health_Risk',
       'Heart_Disease_Risk', 'Age_Group', 'Education_Level', 'Income_Group'],
      dtype='object')


,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,Age,Education,Income,BMI_Category,Healthy_Diet,Mental_Health_Risk,Heart_Disease_Risk,Age_Group,Education_Level,Income_Group
0,0,1,1,20.693674,1,0,0,1,1,1,...,11.725312,6.0,8.0,Normal weight,1,0,2,Young,Some College,High Income
1,0,0,1,28.375985,0,0,0,1,0,1,...,7.977719,5.0,8.0,Obese,0,0,0,Young,Some College,High Income
2,1,1,1,24.629353,0,0,0,1,1,1,...,8.161838,5.0,6.0,Normal weight,1,0,2,Young,Some College,Middle Income
3,0,1,1,27.246610,0,0,0,1,1,1,...,5.038298,6.0,7.0,Overweight,1,0,1,Young,Some College,High Income
4,0,1,1,23.671678,0,0,0,1,1,1,...,12.037360,4.0,6.0,Normal weight,1,0,1,Young,High School Graduate,Middle Income


In [60]:
labels_data_path = 'data/labels.csv'
y = pd.read_csv(labels_data_path)
y = y.loc[:, ~y.columns.str.contains('^Unnamed')]

y.head()

,Diabetes_binary
0,0
1,0
2,1
3,0
4,0


In [61]:
test_data_path = 'data/test.csv'
test_data = pd.read_csv(test_data_path) 
test_data.shape


(50736, 29)

# Preprocessing

In [62]:
#TODO: Preprocessing

# 1) One hot encode variables with string values
# 2) Make correlation matrix to eliminate features with no effect

In [63]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1, random_state=0)
# X_val, X_test, y_val, y_test = model_selection.train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)


In [68]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np

# Separate numerical and categorical columns
categorical_features = X_train.select_dtypes(include=['object']).columns
numerical_features = X_train.select_dtypes(include=['float64', 'int64']).columns

# One-hot encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_cat_encoded = encoder.fit_transform(X_train[categorical_features]).toarray()
X_test_cat_encoded = encoder.transform(X_test[categorical_features]).toarray()

# Normalize numerical features
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train[numerical_features])
X_test_num_scaled = scaler.transform(X_test[numerical_features])

# Combine processed categorical and numerical features
X_train_processed = pd.DataFrame(
    data=np.hstack((X_train_num_scaled, X_train_cat_encoded)),
    columns=list(numerical_features) + list(encoder.get_feature_names_out(categorical_features))
)
X_test_processed = pd.DataFrame(
    data=np.hstack((X_test_num_scaled, X_test_cat_encoded)),
    columns=list(numerical_features) + list(encoder.get_feature_names_out(categorical_features))
)

# Hyperparam sweep

In [65]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score

# Define the hyperparameter grid for the Random Forest
param_grid = {
    'rf__n_estimators': [200, 300],
    'rf__max_depth': [20, 30],
    'rf__min_samples_split': [2, 5, 6],
    'rf__min_samples_leaf': [2, 4, 6],
}

# Perform grid search
model = RandomForestClassifier(random_state=0)

pipe = Pipeline(steps=[
    ('rf', model)
])

grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, scoring='f1', cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_processed, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the model with the best parameters
best_rf = RandomForestClassifier(n_estimators=best_params['rf__n_estimators'], 
                                min_samples_split=best_params['rf__min_samples_split'], 
                                min_samples_leaf=best_params['rf__min_samples_leaf'],
                                max_depth=best_params['rf__max_depth'])
best_rf.fit(X_train_processed, y_train)

# Predict on the test set
y_pred = best_rf.predict(X_test_processed)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')  # Adjust 'binary' for your task
f1 = f1_score(y_test, y_pred, average='binary')                # Adjust 'binary' for your task

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")



Fitting 3 folds for each of 36 candidates, totalling 108 fits


/Users/muriellemardenli/opt/anaconda3/envs/ml_labs/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/muriellemardenli/opt/anaconda3/envs/ml_labs/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/muriellemardenli/opt/anaconda3/envs/ml_labs/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/muriellemardenli/opt/anaconda3/envs/ml_labs/lib/python3.11/site-packages/sklea

KeyboardInterrupt: 

# Model evaluation

In [ ]:
# Test the model on the test set
# y_pred = grid_search.best_estimator_.predict(X_test)
clf = RandomForestClassifier(n_estimators=best_params['rf__n_estimators'], 
                                min_samples_split=best_params['rf__min_samples_split'], 
                                min_samples_leaf=best_params['rf__min_samples_leaf'],
                                max_depth=best_params['rf__max_depth'])

# TODO: Uncomment this line to directly estimate model instead of doing long hyperparameter sweep
# clf = RandomForestClassifier(n_estimators=30, 
#                                 min_samples_split=2, 
#                                 min_samples_leaf=2,
#                                 max_depth=30)

clf.fit(X_train, y_train)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary') # Adjust 'binary' for your task
f1 = f1_score(y_test, y_pred, average='binary')               # Adjust 'binary' for your task

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")

/Users/muriellemardenli/opt/anaconda3/envs/ml_labs/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 0.8656
Precision: 0.5600
F1 Score: 0.2043
